### IMPORTS

In [1]:
import ealib

import yfinance as yf
import logging
import pandas as pd
from collections import Counter
import random
from typing import Tuple

In [2]:
# EDGAR API requires request header
req_header = {"User-Agent": "roberto.brera.24@outlook.com"}

### Logging tests/setup

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)
logging.info("logging info")

2024-07-17 22:46:15,229 - INFO - logging info


### COMP TICKERS

In [4]:
# Dictionary to pandas dataframe
tickers_df = ealib.get_tickers_df(req_header)
tickers_df.head()

2024-07-17 22:46:17,001 - INFO - Request to https://www.sec.gov/files/company_tickers.json returned successfully. Response code: 200


,cik_str,ticker,title
0,0000320193,AAPL,Apple Inc.
1,0000789019,MSFT,MICROSOFT CORP
2,0001045810,NVDA,NVIDIA CORP
3,0001652044,GOOGL,Alphabet Inc.
4,0001018724,AMZN,AMAZON COM INC


In [ ]:
# Examples on searches performable on this dataframe
tickers_df.iloc[133]
tickers_df[tickers_df["ticker"] == "NVDA"]
tickers_df[0:4]

In [ ]:
# Search func examples
ealib.find_ticker(tickers_df, "NVDA")["cik_str"].iloc[0]

# Random companies
tickers_df.iloc[97:111]

In [ ]:
ealib.find_title_substring(tickers_df, "morgan")

### COMP METADATA

In [ ]:
# Find cik for some company given name substring
query_substr =  "gamestop"
query_ticker = ealib.find_title_substring(tickers_df, query_substr)
query_ticker

### Scraping select document types for a company in the tickers dataframe

In [ ]:
# Main parameter setting and fucntion call
comp_out_df = ealib.download_select_filings(
    req_header=req_header, 
    mrps = 8, 
    tickers_df = tickers_df[6000:], 
    root_dir = f"Recent SEC filings for ATMs", 
    query_forms = ["424B5", "S-3"], 
    max_days = 180, 
    write_txt=False, 
    write_pdf=True 
)

comp_out_df

### Debugging section

In [ ]:
query_substr =  "sharplink"
query_ticker = ealib.find_title_substring(tickers_df, query_substr)
query_ticker

### Filtering: Ticker df (yfinance, e.g. marketCap)

In [ ]:
# Suppose we are iterating through tickers_df
curr_ticker = tickers_df.iloc[6849]
curr_yticker = yf.Ticker(curr_ticker["ticker"])

In [ ]:
# Search for particular info
query_info = "cap"
ealib.find_dict_key_substr(curr_yticker.info, query_info)

In [ ]:
# Compute yfinance metric for entire ticker_df
def yf_info(ticker_str, query_info):
    """
    Wrapper function to handle keys-not-found errors,
    and usable with lambda to .apply() to whole series
    """
    ticker_info = yf.Ticker(ticker_str).info
    if query_info not in ticker_info:
        return pd.NA
    return ticker_info [query_info]

# Example using market cap
query_info = "marketCap"
info_series = tickers_df[:1000]["ticker"].apply(lambda x: yf_info(x, query_info))

# How many nans have we got?
info_series.isna().sum()

In [ ]:
# Now apply filtering
market_cap_threshhold = 15*(10**9)
mask = info_series < market_cap_threshhold
filtered_series = info_series[mask]

# How many?
mask.sum()

In [ ]:
# Check other yfinance info, to then compare with SEC
query_info = "operating"
ealib.find_dict_key_substr(curr_yticker.info, query_info)

In [ ]:
query_info = "price"
ealib.find_dict_key_substr(curr_yticker.info, query_info)

### Filtering: Ticker df req--> Comp mtd (e.g. filing)

In [ ]:
# Suppose we are iterating through tickers_df
curr_ticker = tickers_df.iloc[6849]
curr_ticker

In [ ]:
# REQUEST the company dataframe for the iterator ticker
curr_cik = curr_ticker["cik_str"] 
curr_comp_mtd = ealib.get_response_dict(ealib.metadata_url(curr_cik), req_header, mrps=1)

# Extract filing information into dataframe
curr_filings_df = pd.DataFrame.from_dict(curr_comp_mtd["filings"]["recent"])

# Filter the filings for this specific ticker, and for specific filing query 
query_forms = ["424B5", "S-3"]
max_days = 180
curr_select_filings = ealib.filter_filings(curr_filings_df, "filingDate", "form", query_forms, max_days)

# If we find no filings, then the company is excluded
curr_select_filings.empty

### Filtering: Ticker df req--> Company Fact (e.g. oper. cash flow)

In [ ]:
# Suppose we are iterating through tickers_df
curr_ticker = tickers_df.iloc[2098]
curr_ticker

In [ ]:
# Request company facts dictionary
curr_cik = curr_ticker["cik_str"]
"""
TODO: Handle failure for this and other similar requests:
"""
comp_facts = ealib.get_response_dict(ealib.companyfacts_url(curr_cik), req_header, mrps=1)

In [ ]:
# What is the right subdictionary?
""" Most common facts entries by far:
["facts"]["ifrs-full"]
["facts"]["us-gaap"]
"""
comp_facts_subdict = comp_facts["facts"]["us-gaap"]
comp_facts_subdict

In [ ]:
# Script to count different company facts subdictionary entries
tot_tickers = 1000
start_ticker = 8080
keys_counter = Counter()
index_dict = {}
for index, row in tickers_df[start_ticker:start_ticker + tot_tickers].iterrows():
    """ 
    IMPORTANT: Always handle None returns i.e. unsuccessful requests
    """
    cfacts = ealib.get_response_dict(ealib.companyfacts_url(row["cik_str"]), req_header, mrps=8)
    if cfacts == None:
        keys_counter["FAILED_REQS"] += 1
        index_dict["FAILED_REQS"] = index
        continue
    for key in cfacts["facts"].keys():
        keys_counter[key] += 1
        index_dict[key] = index

print(keys_counter)
print(index_dict)
print(f'tot_tickers = {tot_tickers}')
print(f'us-gaap + ifrs + failed reqs = {keys_counter["FAILED_REQS"] + keys_counter["us-gaap"] + keys_counter["ifrs-full"]}')


In [ ]:
# Testing of abstracted Overall function to get a company fact dataframe from an arbitrary ticker
num_tests = 1000
none_rets = 0
for _ in range(num_tests):
    random_number = random.randint(1, 10000)
    res = ealib.company_fact_df(
        tickers_df.iloc[random_number], 
        ["us-gaap", "ifrs-full"], 
        ["NetCashProvidedByUsedInOperatingActivities", "CashFlowsFromUsedInOperatingActivities"], 
        True,
        req_header, 
        8
    )
    # Unpack defensively
    if res is not None:
        res_units, selected_fact, res_df = res
        logging.info(f'company_fact_df selected fact: {selected_fact}')
        """
        print(f'res_units = {res_units}')
        print(res_df)
        """
    else:
        none_rets += 1

print(f'{none_rets} none returns out of {num_tests} tests')

#### Calculating cash burn rate

In [ ]:
# Suppose we are iterating through tickers_df
curr_ticker = tickers_df.iloc[3]
curr_ticker

In [ ]:
# Get the dataframe for some metric
unit, selected_comp_fact, ocf_df = ealib.company_fact_df(
    curr_ticker, 
    ["us-gaap", "ifrs-full"], 
    ["NetCashProvidedByUsedInOperatingActivities", "CashFlowsFromUsedInOperatingActivities"], 
    True,
    req_header, 
    8
)

ocf_df

In [ ]:
filing_date_col = "filed"
form_col = "form"
max_days = 180  # in days

# Use filtering function to filter recent filings and sort list
ocf_df_filt = ealib.filter_filings(ocf_df, filing_date_col, form_col, [""], max_days)
ocf_df_filt

In [ ]:
ealib.ocf_average_daily_burn_rate(ocf_df_filt)

### COMPANY FACTS

In [ ]:
comp_facts = ealib.get_response_dict(ealib.companyfacts_url(query_cik), req_header, mrps=mrps)
us_gaap_facts_dict = comp_facts["facts"]["us-gaap"]
us_gaap_facts_dict.keys()

In [ ]:
comp_facts["facts"]["dei"].keys()

In [ ]:
"""
Shares outstanding
"""
pd.DataFrame(comp_facts["facts"]["dei"]["EntityCommonStockSharesOutstanding"]["units"]["shares"])

In [ ]:

"""
Market cap (proxy)
"""
pd.DataFrame(comp_facts["facts"]["dei"]["EntityPublicFloat"]["units"]["USD"])

In [ ]:

test_qr_substr = "stock"
filtered_keys = find_dict_key_substr(us_gaap_facts_dict, test_qr_substr)

"""
sub_dict = {key: us_gaap_facts_dict[key] for key in filtered_keys if key in us_gaap_facts_dict}
"""
filtered_keys

In [ ]:
us_gaap_facts_dict["CommonStockValue"]["units"]["USD"]

In [ ]:
# Convert to pd data frame
pd.DataFrame(us_gaap_facts_dict["CommonStockValue"]["units"]["USD"])

### COMPANY CONCEPT

In [ ]:
rev_concept = ealib.get_response_dict(
    ealib.companyconcept_url(query_cik, "/us-gaap/Revenues"), 
    req_header,
    mrps=mrps
)

# Coincides with company facts request
pd.DataFrame(rev_concept["units"]["USD"])